## Creating a Classification Model

In this exercise, you will implement a classification model that predicts if a student will score higher than or lower than the median score

### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("GLIM_PGPBABI").enableHiveSupport().getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1485057189851_0005,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [61]:
from pyspark.sql.types import *

from pyspark.sql.functions import *

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

from __future__ import division

### Load Source Data
We will load the studentmaster Hive table into a Saprk Dataframe

In [40]:
student_df = spark.sql("SELECT * FROM studentmaster")

In [41]:
student_df.printSchema()

root
 |-- studentid: integer (nullable = true)
 |-- school: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- addresstype: integer (nullable = true)
 |-- famsize: integer (nullable = true)
 |-- pstatus: integer (nullable = true)
 |-- medu: integer (nullable = true)
 |-- fedu: integer (nullable = true)
 |-- mjob: integer (nullable = true)
 |-- fjob: integer (nullable = true)
 |-- reason: integer (nullable = true)
 |-- guardian: integer (nullable = true)
 |-- traveltime: integer (nullable = true)
 |-- studytime: integer (nullable = true)
 |-- failures: integer (nullable = true)
 |-- schoolsup: integer (nullable = true)
 |-- famsup: integer (nullable = true)
 |-- paid: integer (nullable = true)
 |-- activities: integer (nullable = true)
 |-- nursery: integer (nullable = true)
 |-- higher: integer (nullable = true)
 |-- internet: integer (nullable = true)
 |-- romantic: integer (nullable = true)
 |-- famrel: integer (nullable = true)
 |-

### Prepare the Data
Most modeling begins with exhaustive exploration and preparation of the data. In this example, the data has been cleaned for you. You will select all the columns to use as *features* and create a Boolean *label* field named **BelowMedian** with the value **1** for students scoring less than the median, or **0** if they score greater than or equal to the median score.

(Note that in a real scenario, you would perform additional tasks such as handling missing or duplicated data, scaling numeric columns, and using a process called *feature engineering* to create new features for your model).

In [42]:
medianScore = student_df.approxQuantile("g3", [0.5], 0.25)[0]

In [43]:
print "The median final score is %.2f" %medianScore

The median final score is 10.00

In [44]:
#we no longer require with the smoteclass column
data = student_df.drop('smoteclass').withColumn("BelowMedian", (col('g3') < medianScore).cast("Int"))

In [45]:
#let's check the summary stats for the continuous and categorical target variables
data.select('g3','BelowMedian').describe().show()

+-------+------------------+-------------------+
|summary|                g3|        BelowMedian|
+-------+------------------+-------------------+
|  count|              2320|               2320|
|   mean|11.306034482758621|0.21379310344827587|
| stddev|3.1378227577428275|0.41007084059151344|
|    min|                 0|                  0|
|    max|                19|                  1|
+-------+------------------+-------------------+

In [46]:
#as this is a classification problem we will go ahead and drop the g3 column as well
data = data.drop('g3')

In [47]:
data.count()

2320

In [48]:
data.printSchema()

root
 |-- studentid: integer (nullable = true)
 |-- school: integer (nullable = true)
 |-- sex: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- addresstype: integer (nullable = true)
 |-- famsize: integer (nullable = true)
 |-- pstatus: integer (nullable = true)
 |-- medu: integer (nullable = true)
 |-- fedu: integer (nullable = true)
 |-- mjob: integer (nullable = true)
 |-- fjob: integer (nullable = true)
 |-- reason: integer (nullable = true)
 |-- guardian: integer (nullable = true)
 |-- traveltime: integer (nullable = true)
 |-- studytime: integer (nullable = true)
 |-- failures: integer (nullable = true)
 |-- schoolsup: integer (nullable = true)
 |-- famsup: integer (nullable = true)
 |-- paid: integer (nullable = true)
 |-- activities: integer (nullable = true)
 |-- nursery: integer (nullable = true)
 |-- higher: integer (nullable = true)
 |-- internet: integer (nullable = true)
 |-- romantic: integer (nullable = true)
 |-- famrel: integer (nullable = true)
 |-

### Split the Data
It is common practice when building supervised machine learning models to split the source data, using some of it to train the model and reserving some to test the trained model. In this exercise, you will use 70% of the data for training, and reserve 30% for testing.

In [49]:
splits = data.randomSplit([0.8, 0.2])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print "Training Rows:", train_rows, " Testing Rows:", test_rows

Training Rows: 1831  Testing Rows: 489

### Prepare the Training Data
To train the classification model, you need a training data set that includes a vector of numeric features, and a label column. In this exercise, you will use the **VectorAssembler** class to transform the feature columns into a vector, and then rename the **Late** column to **label**.

In [50]:
inputs = train.columns

In [51]:
inputs.remove('BelowMedian')

In [52]:
inputs

['studentid', 'school', 'sex', 'age', 'addresstype', 'famsize', 'pstatus', 'medu', 'fedu', 'mjob', 'fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'dalc', 'walc', 'health', 'absences', 'g1', 'g2']

In [53]:
assembler = VectorAssembler(inputCols = inputs, outputCol="features")
training = assembler.transform(train).select(col("features"), col("BelowMedian").alias("label"))
training.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.0,0.0,0.0,15.0...|    0|
|[4.0,0.0,0.0,17.0...|    0|
|[5.0,0.0,0.0,15.0...|    0|
|[8.0,0.0,0.0,16.0...|    0|
|[9.0,0.0,0.0,16.0...|    0|
|[10.0,0.0,0.0,16....|    0|
|[11.0,0.0,0.0,15....|    0|
|[12.0,0.0,0.0,15....|    0|
|[14.0,0.0,0.0,16....|    0|
|[16.0,0.0,0.0,16....|    0|
|[17.0,0.0,0.0,16....|    0|
|[18.0,0.0,0.0,16....|    0|
|[19.0,0.0,0.0,16....|    0|
|[20.0,0.0,0.0,16....|    0|
|[22.0,0.0,0.0,16....|    0|
|[23.0,0.0,0.0,15....|    0|
|[24.0,0.0,0.0,15....|    0|
|[25.0,0.0,0.0,15....|    0|
|[26.0,0.0,0.0,16....|    0|
|[27.0,0.0,0.0,15....|    0|
+--------------------+-----+
only showing top 20 rows

### Train a Classification Model
Next, you need to train a classification model using the training data. To do this, create an instance of the classification algorithm you want to use and use its **fit** method to train a model based on the training DataFrame. In this exercise, you will use a *Logistic Regression* classification algorithm - though you can use the same technique for any of the classification algorithms supported in the spark.ml API.

In [54]:
lr = LogisticRegression(labelCol="label",featuresCol="features",maxIter=10,regParam=0.3)
model = lr.fit(training)
print "Model trained!"

Model trained!

### Prepare the Testing Data
Now that you have a trained model, you can test it using the testing data you reserved previously. First, you need to prepare the testing data in the same way as you did the training data by transforming the feature columns into a vector. This time you'll rename the **BelowMedian** column to **trueLabel**.

In [56]:
testing = assembler.transform(test).select(col("features"), col("BelowMedian").alias("trueLabel"))
testing.show()

+--------------------+---------+
|            features|trueLabel|
+--------------------+---------+
|(33,[3,4,7,8,10,1...|        0|
|[1.0,0.0,0.0,17.0...|        0|
|[3.0,0.0,0.0,16.0...|        0|
|[6.0,0.0,0.0,15.0...|        0|
|[7.0,0.0,0.0,16.0...|        0|
|[13.0,0.0,0.0,15....|        0|
|[15.0,0.0,0.0,15....|        0|
|[21.0,0.0,0.0,16....|        0|
|[30.0,0.0,0.0,16....|        0|
|[33.0,0.0,0.0,15....|        0|
|[43.0,0.0,0.0,15....|        0|
|[45.0,0.0,0.0,16....|        0|
|[57.0,0.0,0.0,16....|        0|
|[58.0,0.0,0.0,16....|        0|
|[60.0,0.0,0.0,16....|        0|
|[66.0,0.0,0.0,16....|        0|
|[68.0,0.0,0.0,17....|        1|
|[70.0,0.0,0.0,16....|        0|
|[72.0,0.0,0.0,19....|        0|
|[76.0,0.0,0.0,17....|        0|
+--------------------+---------+
only showing top 20 rows

### Test the Model
Now you're ready to use the **transform** method of the model to generate some predictions. You can use this approach to predict the final score category; but in this case you are using the test data which includes a known true label value, so you can compare the predicted status to the actual status. 

In [57]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "probability", "trueLabel")
predicted.show(100)

+--------------------+----------+--------------------+---------+
|            features|prediction|         probability|trueLabel|
+--------------------+----------+--------------------+---------+
|(33,[3,4,7,8,10,1...|       0.0|[0.63843592166474...|        0|
|[1.0,0.0,0.0,17.0...|       0.0|[0.86022864579177...|        0|
|[3.0,0.0,0.0,16.0...|       0.0|[0.93660209702692...|        0|
|[6.0,0.0,0.0,15.0...|       0.0|[0.94469176957182...|        0|
|[7.0,0.0,0.0,16.0...|       0.0|[0.97540671788211...|        0|
|[13.0,0.0,0.0,15....|       0.0|[0.93062610852263...|        0|
|[15.0,0.0,0.0,15....|       0.0|[0.97530058248934...|        0|
|[21.0,0.0,0.0,16....|       0.0|[0.94751440216820...|        0|
|[30.0,0.0,0.0,16....|       0.0|[0.95158088118332...|        0|
|[33.0,0.0,0.0,15....|       0.0|[0.94171594886607...|        0|
|[43.0,0.0,0.0,15....|       0.0|[0.96504810914054...|        0|
|[45.0,0.0,0.0,16....|       0.0|[0.82722652680658...|        0|
|[57.0,0.0,0.0,16....|   

Looking at the result, the **prediction** column contains the predicted value for the label, and the **trueLabel** column contains the actual known value from the testing data. It looks like there are a mix of correct and incorrect predictions - later in this course you'll learn how to measure the accuracy of a model.

In [82]:
testObs = testing.count()*1.0

In [70]:
accuratePredictions = prediction.select("features", "prediction", "probability", "trueLabel").filter(
    prediction['prediction'] == prediction['trueLabel']).count()

In [96]:
print "The Accuracy rate over all predictions is %.3f %%" %((accuratePredictions/testObs) *100)

The Accuracy rate over all predictions is 82.004 %